## 准备数据

In [87]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [88]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [89]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(shape=[28*28, 256], dtype=tf.float32,initial_value=tf.random.uniform(shape=[28*28, 256],
                              minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[256], dtype=tf.float32, initial_value=tf.zeros(256))
        self.W2 = tf.Variable(shape=[256, 16], dtype=tf.float32,initial_value=tf.random.uniform(shape=[256, 16],
                              minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[16], dtype=tf.float32, initial_value=tf.zeros(16))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [90]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [91]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.8543417 ; accuracy 0.040516667
epoch 1 : loss 2.8258078 ; accuracy 0.048
epoch 2 : loss 2.798144 ; accuracy 0.056283332
epoch 3 : loss 2.7713015 ; accuracy 0.06428333
epoch 4 : loss 2.745235 ; accuracy 0.07328334
epoch 5 : loss 2.7199013 ; accuracy 0.08268333
epoch 6 : loss 2.6952624 ; accuracy 0.09201667
epoch 7 : loss 2.6712816 ; accuracy 0.10205
epoch 8 : loss 2.647925 ; accuracy 0.11285
epoch 9 : loss 2.6251621 ; accuracy 0.12415
epoch 10 : loss 2.6029632 ; accuracy 0.13621667
epoch 11 : loss 2.5813015 ; accuracy 0.14881666
epoch 12 : loss 2.5601518 ; accuracy 0.16171667
epoch 13 : loss 2.5394902 ; accuracy 0.17471667
epoch 14 : loss 2.5192943 ; accuracy 0.18848333
epoch 15 : loss 2.4995444 ; accuracy 0.20343333
epoch 16 : loss 2.4802206 ; accuracy 0.21826667
epoch 17 : loss 2.4613042 ; accuracy 0.2329
epoch 18 : loss 2.4427786 ; accuracy 0.24668333
epoch 19 : loss 2.424628 ; accuracy 0.26113334
epoch 20 : loss 2.4068372 ; accuracy 0.27498335
epoch 21 : loss 2.3893